Vanilla naive bayes mean it is from scratch without any optimization. Just the main algorithm 

In [30]:
import nltk
#nltk.download('stopwords')
#nltk.download('twitter_samples')

In [31]:
from utils import process_tweet, lookup
from nltk.corpus import twitter_samples
import numpy as np
import nltk

from os import getcwd

In [85]:
filepath = f"C:/Users/user/AppData/Roaming/nltk_data/corpora"

nltk.data.path.append(filepath)

In [86]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [87]:
# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

#concat the pos and neg to create the total dataset
train_x = train_pos + train_neg
test_x = test_pos + test_neg


train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [48]:
class Vanila_NB:
    
    def __init__(self, result, tweets, ys, freq, train_x, train_y, tweet, logprior, likelihood, test_x, test_y):
        self.result = result
        self.tweets = tweets
        self.ys = ys
        self.freq = freq
        self.train_x = train_x
        self.train_y = train_y
        self.tweet = tweet
        self.logprior = logprior
        self.likelihood = likelihood
        self.test_x = test_x
        self.test_y = test_y
        
    def count_tweets(result, tweets, ys):

        for y, tweet in zip(ys, tweets):
            for word in process_tweet(tweet):
                # define the key, which is the word and label tuple
                pair = (word, y)

                # if the key exists in the dictionary, increment the count
                if pair in result:
                    result[pair] += 1

                # else, if the key is new, add it to the dictionary and set the count to 1
                else:
                    result[pair] = 1


        return result
    
    
    
    def train_naive_bayes(freqs, train_x, train_y):
        '''
        Input:
            freqs: dictionary from (word, label) to how often the word appears
            train_x: a list of tweets
            train_y: a list of labels correponding to the tweets (0,1)
        Output:
            logprior: the log prior. (equation 3 above)
            loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
        '''
        loglikelihood = {}
        logprior = 0



        # calculate V, the number of unique words in the vocabulary
        vocab = set([pair[0] for pair in freqs.keys()])
        V = len(vocab)

        # calculate N_pos and N_neg
        N_pos = N_neg = 0
        for pair in freqs.keys():
            # if the label is positive (greater than zero)
            if pair[1] > 0:

                # Increment the number of positive words by the count for this (word, label) pair
                N_pos += freqs[pair]

            # else, the label is negative
            else:

                # increment the number of negative words by the count for this (word,label) pair
                N_neg += freqs[pair]

        # Calculate D, the number of documents
        D = len(train_y)

        # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
        D_pos = sum(train_y)

        # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
        D_neg = D - D_pos

        # Calculate logprior
        logprior = np.log(D_pos) - np.log(D_neg)

        # For each word in the vocabulary...
        for word in vocab:
            # get the positive and negative frequency of the word
            freq_pos = lookup(freqs, word, 1)
            freq_neg = lookup(freqs, word, 0)

            # calculate the probability that each word is positive, and negative
            p_w_pos = (freq_pos + 1) / (N_pos + V)
            p_w_neg = (freq_neg + 1) / (N_neg + V)

            # calculate the log likelihood of the word
            loglikelihood[word] = np.log(p_w_pos / p_w_neg)


        return logprior, loglikelihood
    
    def naive_bayes_predict(tweet, logprior, loglikelihood):
        '''
        Input:
            tweet: a string
            logprior: a number
            loglikelihood: a dictionary of words mapping to numbers
        Output:
            p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

        '''

        # process the tweet to get a list of words
        word_l = process_tweet(tweet)

        # initialize probability to zero
        p = 0

        # add the logprior
        p += logprior

        for word in word_l:

            # check if the word exists in the loglikelihood dictionary
            if word in loglikelihood:
                # add the log likelihood of that word to the probability
                p += loglikelihood[word]



        return p
    
    
    def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
        """
        Input:
            test_x: A list of tweets
            test_y: the corresponding labels for the list of tweets
            logprior: the logprior
            loglikelihood: a dictionary with the loglikelihoods for each word
        Output:
            accuracy: (# of tweets classified correctly)/(total # of tweets)
        """
        accuracy = 0  # return this properly
        
        

        y_hats = []
        for tweet in test_x:
            # if the prediction is > 0
            if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
                # the predicted class is 1
                y_hat_i = 1
            else:
                # otherwise the predicted class is 0
                y_hat_i = 0

            # append the predicted class to the list y_hats
            y_hats.append(y_hat_i)

        # error is the average of the absolute values of the differences between y_hats and test_y
        error = np.sum(np.abs(y_hats - test_y))/len(test_y)

        # Accuracy is 1 minus the error
        accuracy = 1 - error



        return accuracy

In [5]:
def count_tweets(result, tweets, ys):

    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            # define the key, which is the word and label tuple
            pair = (word, y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1


    return result

In [6]:
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

In [36]:
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
Vanila_NB.count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

In [7]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
    '''
    loglikelihood = {}
    logprior = 0



    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos and N_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:

            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += freqs[pair]

        # else, the label is negative
        else:

            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs[pair]

    # Calculate D, the number of documents
    D = len(train_y)

    # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
    D_pos = sum(train_y)

    # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
    D_neg = D - D_pos

    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos / p_w_neg)


    return logprior, loglikelihood

In [8]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''

    # process the tweet to get a list of words
    word_l = process_tweet(tweet)

    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior

    for word in word_l:

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]



    return p

In [9]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly

    
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.sum(np.abs(y_hats - test_y))/len(test_y)

    # Accuracy is 1 minus the error
    accuracy = 1 - error

    

    return accuracy

In [43]:
# Build the freqs dictionary for later uses
freqs = Vanila_NB.count_tweets({}, train_x, train_y)

In [44]:
logprior, loglikelihood = Vanila_NB.train_naive_bayes(freqs, train_x, train_y)
print(logprior) # you can print this out
print(len(loglikelihood)) # you can print the length of this one

0.0
9091


In [47]:
print("Naive Bayes accuracy = %0.4f" %(test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

NameError: name 'naive_bayes_predict' is not defined

In [49]:
# Experiment with your own tweet.
my_tweet = 'She smiled.'
p = Vanila_NB.naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 1.5800735036549516


In [50]:
my_tweet = 'She stole.'
p = Vanila_NB.naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is -1.0871547029270034


In [78]:
import re, string, nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [51]:
import numpy as np

In [94]:
class NaiveBayes:
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y
        self.freqs = self.__count_tweets(self.train_x, self.train_y)
        self.loglikelihood = {}
        self.logprior = 0


    def train(self):
        '''
        Input:
            freqs: dictionary from (word, label) to how often the word appears
            train_x: a list of tweets
            train_y: a list of labels correponding to the tweets (0,1)
        Output:
            logprior: the log prior. (equation 3 above)
            loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)

        '''


        # calculate V, the number of unique words in the vocabulary
        vocab = set([pair[0] for pair in self.freqs.keys()])
        V = len(vocab)

        # calculate N_pos and N_neg
        N_pos = N_neg = 0
        for pair in self.freqs.keys():
            # if the label is positive (greater than zero)
            if pair[1] > 0:

                # Increment the number of positive words by the count for this (word, label) pair
                N_pos += self.freqs[pair]

            # else, the label is negative
            else:

                # increment the number of negative words by the count for this (word,label) pair
                N_neg += self.freqs[pair]

        # Calculate D, the number of documents
        D = len(self.train_y)

        # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
        D_pos = sum(self.train_y)

        # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
        D_neg = D - D_pos

        # Calculate logprior
        self.logprior = np.log(D_pos) - np.log(D_neg)

        # For each word in the vocabulary...
        for word in vocab:
            # get the positive and negative frequency of the word
            freq_pos = self.__lookup(self.freqs, word, 1)
            freq_neg = self.__lookup(self.freqs, word, 0)

            # calculate the probability that each word is positive, and negative
            p_w_pos = (freq_pos + 1) / (N_pos + V)
            p_w_neg = (freq_neg + 1) / (N_neg + V)

            # calculate the log likelihood of the word
            self.loglikelihood[word] = np.log(p_w_pos / p_w_neg)


        return logprior, loglikelihood

    def predict(self, tweet):
        if len(self.loglikelihood) < 1:
            print("You have to train the model before trying to predict...")
            print("Or you can call the load method to load a model file")
        else:
            word_l = self.__process_tweet(tweet)
            p = 0
            p += self.logprior
            for word in word_l:
                if word in self.loglikelihood:
                    p += self.loglikelihood[word]

            return p
    
    def test(self, test_x, test_y):
        """
        Input:
            test_x: A list of tweets
            test_y: the corresponding labels for the list of tweets
            logprior: the logprior
            loglikelihood: a dictionary with the loglikelihoods for each word
        Output:
            accuracy: (# of tweets classified correctly)/(total # of tweets)
        """
        accuracy = 0  # return this properly


        y_hats = []
        for tweet in test_x:
            # if the prediction is > 0
            if self.predict(tweet, self.logprior, self.loglikelihood) > 0:
                # the predicted class is 1
                y_hat_i = 1
            else:
                # otherwise the predicted class is 0
                y_hat_i = 0

            # append the predicted class to the list y_hats
            y_hats.append(y_hat_i)

        # error is the average of the absolute values of the differences between y_hats and test_y
        error = np.sum(np.abs(y_hats - test_y))/len(test_y)

        # Accuracy is 1 minus the error
        accuracy = 1 - error

    

        return accuracy


    def save(self):
        if len(self.loglikelihood) < 1:
            print("You have to train the model before saving it...")
            print("Or you can call the load method to load a model file")
        else:
            total_objects = {
                'loglikelihood': self.loglikelihood,
                'logprior': self.logprior
            }

            with open('model.json', 'w') as file:
                json.dump(total_objects, file)

            print("Model saved successfully")

    def load(self):
        try:
            with open(file_path, 'r') as file:
                model_parameters = json.load(file)
            self.loglikelihood = model_parameters['loglikelihood']
            self.logprior = model_parameters['logprior']

            print("Model loaded successfully")
        except:
            print("Please provide a valid filepath, the file should be a json file")
    
    def __process_tweet(self, tweet):
        stemmer = PorterStemmer()
        stopwords_english = stopwords.words('english')

        # remove stock market tickers like $GE
        tweet = re.sub(r'\$\w*', '', tweet)

        # remove old style retweet text "RT"
        tweet = re.sub(r'^RT[\s]+', '', tweet)

        # remove hyperlinks
        tweet = re.sub(r'https?:\/\/.[\r\n]', '', tweet)

        # remove hashtags
        # only removing the hash # sign from the word
        tweet = re.sub(r'#', '', tweet)

        # tokenize tweets
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                reduce_len=True)
                                
        tweet_tokens = tokenizer.tokenize(tweet)

        tweets_clean = []
        whitelist = ["n't", "not", "no"]
        for word in tweet_tokens:
            if ((word not in stopwords_english or word in whitelist) and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
                stem_word = stemmer.stem(word)  # stemming word
                tweets_clean.append(stem_word)

        return tweets_clean



    def __count_tweets(self, tweets, ys):
        
        result = {}
        for y, tweet in zip(ys, tweets):
            
            for word in process_tweet(tweet):
                # define the key, which is the word and label tuple
                pair = (word, y)

                # if the key exists in the dictionary, increment the count
                if pair in result:
                    result[pair] += 1

                # else, if the key is new, add it to the dictionary and set the count to 1
                else:
                    result[pair] = 1

        return result

def __lookup(self, freqs, word, label):

        n = 0
        pair = (word, label)
        if (pair in freqs):
            n = freqs[pair]

        return n

In [95]:
nb = NaiveBayes(train_x, train_y)

In [96]:
nb.predict("She a good person")

You have to train the model before trying to predict...
Or you can call the load method to load a model file
